In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sympy
from sympy import symbols

import warnings
warnings.simplefilter('ignore')

# Introduction/Basics of futures markets (OFOD CH01, 02)

## Introduction

### History

- In 1848, the first derivatives exchange was created in Chicago, United
States for commodity markets (The Chicago Board of Trade-CBOT)
- The CME was created in 1874

### Amount outstanding

OTC is growing.

### Definition 

- A derivatives is a financial security with a value derived from an underlying asset
- Different types of risks can be managed with derivatives, for example : stocks, currencies, interest rates, commodities, debt instruments, volatility, electricity, insurance payouts, default, weather, etc
- Derivatives play a key role in transferring risks in the economy (hedging) but they could be also used for speculation


### Main products

- A futures/forward contract is an agreement to buy or sell an asset for a certain price at a certain time
- A swap is an agreement to exchange cash-flows in the future
- An option is a right to buy (call) or sell (put) a certain asset by (American style) or at (European style) a certain date for a certain price to the other

### How it works

Derivatives are
- contracts between two parties with a defined lifetime: to every long position, there is an outstanding short position
- a zero-sum game : at any time, the cash-flows received (paid) by the holder of a derivative are paid (received) by his counterpart

### Markets

- Exchange traded
- OTC

### Key players

- Hedgers: seek to hedge or reduce their risks with derivatives (interest rate risk, exchange rate risk, market risk)
- Speculators: use derivatives to take a position in the market, betting that the underlying value will increase or decrease
- Arbitrageurs: take advantage of abnormal differences in the price of different assets
and lock-in riskless profits at no cost

## Basics of Futures and Forward markets

### Forwards and Futures definition

A forward contract is:

- an OTC contract between two parties to buy or sell an asset for a certain price at a certain time in the future
- all terms (forward price, underlying asset, size, delivery date, delivery place) can be negotiated (customised)
- the price at which the trade will occur is locked in today but there are no intermediate cash-flows
- at maturity
 - the buyer (holding a long position) has the obligation to pay the forward price against the delivery of the asset
 - the seller (holding a short position) has the obligation to deliver the asset and receive the forward price in exchange

Example of Forward contract:
- Two parties agree today to exchange 500 000 barrels of crude oil – quality WTI (West Texas Intermediate) - for USD 42.08 a barrel in three months from today, with a delivery in Cushing (Oklahoma-USA).
- Terms : forward price (42,08), underlying asset (WTI), size (500000 barrels), delivery date (T+3M), delivery place (Cushing) 

A futures contract is:
- an exchange traded contract to deliver (in a case of a short position) or to receive (in a case of a long position, counterpart) a given quantity of a given asset at a future date at a price fixed today (futures price)
- standardized : all contracts are identical (they are fungible) in terms of underlying asset (commodities, financial assets), contract size, delivery arrangements, or maturity (delivery month)

Example of Futures contract:

- WTI futures listed on the CME (Chicago Mercantile Exchange)
- Underlying : WTI crude oil
- contract size : 1 000 barrels (42,000 US gallons)
- delivery : at any pipeline or storage facility in Cushing
- maturity : 72 consecutive months are listed
- quotation: US dollars per barrel

### Trading in futures contracts

#### PL for long / short

In [2]:
# Function of PL for long 
def PL_long(n_of_contracts, nominal_value_futures, long_entry_price, closed_out_price):
    PL_long = (closed_out_price - long_entry_price) * nominal_value_futures * n_of_contracts
    return PL_long
## USD

In [3]:
# Example with long position of crude oil WTI futures
# Given by questions
n_of_contracts = 10
nominal_value_futures = 1000
long_entry_price = 40
closed_out_price = 50

# Impute the given values and get the answer
PL_long = PL_long(n_of_contracts, nominal_value_futures, long_entry_price, closed_out_price)
PL_long

100000

In [4]:
# Function of PL for short
def PL_short(n_of_contracts, nominal_value_futures, short_entry_price, closed_out_price):
    PL_short = (short_entry_price - closed_out_price) * nominal_value_futures * n_of_contracts
    return PL_short
## USD

In [5]:
# Example with long position of crude oil WTI futures
# Given by questions
n_of_contracts = 10
nominal_value_futures = 1000
short_entry_price = 45
closed_out_price = 44

# Impute the given values and get the answer
PL_short = PL_short(n_of_contracts, nominal_value_futures, short_entry_price, closed_out_price)
PL_short

10000

#### Margin call calculation

In [6]:
# Function of margin call calculation
def margin_call_3days(n_of_contracts, contract_size, futures_price, marginal_requirement, maintenance_margin, settlement_price_firstDay, settlement_price_secondDay, settlement_price_thirdDay):
    # First day settlement
    marginal_account = marginal_requirement - (futures_price - settlement_price_firstDay) * n_of_contracts * contract_size
    if marginal_account < maintenance_margin:
        marginal_account = marginal_requirement
    else:
        marginal_account = marginal_account
    
    # Second day settlement
    marginal_account = marginal_account - (settlement_price_firstDay - settlement_price_secondDay) * n_of_contracts * contract_size
    if marginal_account < maintenance_margin:
        marginal_account = marginal_requirement
    else:
        marginal_account = marginal_account
    
    # Third day settlement
    marginal_account = marginal_account - (settlement_price_secondDay - settlement_price_thirdDay) * n_of_contracts * contract_size
    if marginal_account < maintenance_margin:
        marginal_account = marginal_requirement
    else:
        marginal_account = marginal_account
    
    return marginal_account

In [7]:
# Example of margin call calculation

# Given by questions
n_of_contracts = 2
contract_size = 100 ## oz
futures_price = 1250 ## USD per oz
mr = 6000 ## USD
marginal_requirement = mr * n_of_contracts
mm = 4500 ## USD
maintenance_margin = mm * n_of_contracts
settlement_price_firstDay = 1241 ## USD
settlement_price_secondDay = 1231 ## USD
settlement_price_thirdDay = 1251 ## USD

answer = margin_call_3days(n_of_contracts, contract_size, futures_price, marginal_requirement, maintenance_margin, settlement_price_firstDay, settlement_price_secondDay, settlement_price_thirdDay)
answer ## USD

16000

### Forwards VS Futures

Forwards:
- Private contract between 2 parties
- Fully negociable contract
- Usually 1 specified delivery date
- Settled at end of contract
- Delivery or final cash settlement usually occurs
- Some credit risk

Futures:
- Exhange traded
- Standard contract
- Range of delivery dates
- Settled daily
- Contract usually closed out prior to maturity
- Virtally no credit risk

# Option fundamentals (OFOD CH10)

Outline:
- Definition
- Type of options
- Positions, payoffs, P/L
- Underlying assets and specifications
- Trading

## Definition
- An option is a right to buy (call) or sell (put) an underlying asset at a specified price (strike price) within a specific timeframe.
- The price of an option is called a premium
- Total value of an option (premium) = the intrinsic value + the time value

## Type of options

Call options
- a European call option gives its holder (who has a long position) the right to buy a certain asset at a certain date for a certain price (the strike price)

Put options
- a European put option gives its holder (who has a long position) the right to sell a certain asset at a certain date for a certain price (the strike price)

American options vs. European options
- an American option can be exercised at any time during its life
- when the option is exercised before maturity, we speak of early exercise
- even American options can only be exercised once

## Positions, payoffs, P/L
The 4 basic possible positions summarized

long call
- pay a premium today for the right to buy in the future
- P/LT = max(ST – K; 0) – c = (ST – K) – c

long put
- pay a premium today for the right to sell in the future
- P/LT = max(K – ST; 0) – p

short call
- receive a premium today against the possibility of having to sell in the future
- P/LT = –max(ST – K; 0) + c

short put
- receive a premium today against the possibility of having to buy in the future
- P/LT = –max(K – ST; 0) + p

## Underlying assets and specifications

Types of underlying
- Stocks: premium are quoted for 1 stock, contract stands for a given quantity of stocks (100, for example)
- currencies: premium given in cents per unit of foreign currency, contract on a certain amount of foreign currency (€125,000)
- indices: exclusively cash-settled
- interest rates: mostly short-terms maturities
- futures

Dividends, Stock split

## Trading

- Option class
- Option series
- At-the-money option/In-the-money option/Out-of-the money option
- Moneyness
- The intrinsic value is the maximum of zero and the value the option would have should it be exercised immediately
 - Call : VI_t = max(S_t – K ; 0 )
 - Put : VI_t = max(K – S_t; 0 )
- The time value is the difference between the total value of an option (premium) and its intrinsic value
 - Call : VS_t = c_t – VI_t
 - Put : VS_t = p_t – VI_t

## Exercise

### Exercise 1 <br>

An investor buys a stock at 14 USD per share.

He assumes that the price will go up, but in the event that the stock value goes down, he buys a put option at a premium of 1 USD with a strike at 10 USD with a 6-month maturity.

What happens at expiration if:
- the value of the stock has increased to 16 USD ?
- the value of the stock has decreased to 8 USD ?

Answer

If the value of the stock has increased to 16 USD, <br>
He will not exercise the put option and will lost the premium (1 USD)

If the value of the stock has decreased to 8 USD, <br>
He can sell the stock he bought (at 14 USD per share) for 10 USD per share. With the put option, he limited his losses to 4 USD per share (without taking into account the premium of 1 USD paid). Without the put option, he would have lost 6 USD per share.

### Exercise 2
A speculator purchased a call option on British pounds for USD 0.02 per unit. The
strike price was USD 1.45 and the spot rate at the time the option was exercised
was USD 1.46
Assume there are 31,250 units in a British pound option. What was his net P/L
on this option?

Answer

- P/L per unit on exercising the option = USD 0.01
- Premium paid per unit = USD 0.02
- Net P/L per unit = – USD 0.01
- Net P/L per option = 31,250 units × (–USD 0.01) = –USD 312.50

# Arbitrage relationships (OFOD CH11)

## Arbitrage relationship

- is a model-free restriction imposed on the value of options that prevents arbitrage opportunities
- When such a condition is not met 
 - it is possible to build a portfolio that generates arbitrage profits

How can we demonstrate an arbitrage relationship?

 1. we first assume the relationship does not hold
 2. then we build the arbitrage portfolio
  - with positive or zero initial payoffs
  - with positive or zero future payoffs at all dates before expiration and at expiration (take care about possible early exercise of American options)

The idea is simply that
 - if, for certain prices, an arbitrage exists, these prices are not compatible with an equilibrium

## Minimum value

- The option premium must be higher than the greater of zero and
 - the difference between the underlying value (S) and the present value of the strike price (K) for the European call (c)
 - the difference between the present value of the strike price (K) and the underlying price (S) for the European put (p)

- Any option price below its minimum value should trigger immediate arbitrage trades

c >= max(S_0 - D - K * e ** (-r * T) ; 0) <br>
p >= max(K * e ** (-r * T) + D - S_0 ; 0)

In [ ]:
# Function of theoretical call price
def call_theoretical_price(S_0, K, r, T, D):
    call_theoretical_price = np.maximum(S_0 - D - K * np.exp(-r*T), 0)
    return call_theoretical_price

In [ ]:
# Function of call arbitrage judgements
def call_arbitrage(S_0, K, r, T, theoretical_c, c, S_t):
    if c < theoretical_c:
        print("Yes.")
        print("A trader can short the stock " + str(S_0) + " USD for " + str(T) + " and buy the call to provide a cash inflow of " 
              + str(S_0 - c)
              + " USD. If he invests the proceeds during " + str(T) + " yrs at " + str(r) + " per year, " 
              + str(S_0 - c) + " USD becomes " + str((S_0 - c) * np.exp(r * T)) + " USD.")
        a = (S_0 - c) * np.exp(r * T)
        if S_t > K:
            print("A trader gets " + str(a - K) + " USD!")
        else:
            print("A trader sells the stock and gets " + str(a - S_t) + " USD!")
    else:
        print("No.")

In [ ]:
# Example for call
# Given the questions
S_0 = 20 ## USD
K = 18 ## USD
r = 0.1 ## % per year
T = 1 ## yr
D = 0 ## 0 when no divident

# Call Theoretical Price 
theoretical_c = call_theoretical_price(S_0, K, r, T, D)
theoretical_c

# A trader can get a profit from arbitrage or not
c = 3 ## USD
S_t = 20

call_arbitrage(S_0, K, r, T, theoretical_c, c, S_t)

In [ ]:
# Function of theoretical put price
def put_theoretical_price(S_0, K, r, T, D):
    put_theoretical_price = np.maximum(K * np.exp(-r*T) + D - S_0, 0)
    return put_theoretical_price

In [ ]:
# Function of put arbitrage judgements
def put_arbitrage(S_0, K, r, T, theoretical_p, p, S_t):
    if p < theoretical_p:
        print("Yes.")
        print("A trader can borrow the money " + str(S_0 + p) + " USD for " + str(T) + " years in order to buy the put " + str(p)
              + " USD and the stock " + str(S_0) + " USD. At the end, the trader should pay " + str((S_0 + p) * np.exp(r*T))
              + " USD.")
        a = (S_0 + p) * np.exp(r*T)
        if S_t < K:
            print("A trader exercise the option and gets " + str(K - a) + " USD!")
        else:
            print("A trader sells the stock and gets " + str(S_t - a) + " USD!")
    else:
        print("No.")

In [ ]:
# Example for put
# Given the questions
S_0 = 37 ## USD
K = 40 ## USD
r = 0.05 ## % per year
T = 0.5 ## yr
D = 0 ## 0 when no divident

# put Theoretical Price 
theoretical_p = put_theoretical_price(S_0, K, r, T, D)
theoretical_p

# A trader can get a profit from arbitrage or not
p = 1 ## USD
S_t = 38

put_arbitrage(S_0, K, r, T, theoretical_p, p, S_t)

## Constraints on options prices related to strikes

Conditions on put vertical spreads (K1 < K2)
- the value of a put must always be greater than the value of an identical put with a lower strike price

p(K2) > p(K1)

- the difference in the value of two puts with different strike prices cannot be greater than the difference in strike prices

p(K2) - p(K1) <= K2 - K1

## Put call parity

The two portfolios are equal in the final P/L;

Portfolio 1:
- Long 1 call (c)
- Cash Investment (K * e**(-r * T)

Porfolio 2:
- Long 1 put (p)
- Long 1 share (S_t)

When S_t <= K,both have the same P/L: K. <br>
When S_t > K, both have the same P/L: S_t

So, the initial values of both portfolios should be the same as well. <br>
PCP: <br>
c + K * e**(-r * T) = p + S_0

This can be rewritten as:
c_PCP = p + S_0 - K * e**(-r * T)

In [ ]:
# Function of PCP
def PCP(S_0, r, c, K, T, D, p, S_t):
    c_PCP = p + S_0 - D - K * np.exp(-r * T)
    if c_PCP > c:
        print("The call is undervalued.")
        print("Action now!")
        print("Buy the call " + str(c) + " USD and short the put " + str(p) + " USD, short the stock " 
              + str(S_0) + " USD and invest the proceeds " + str(-c + p + S_0) + " USD for 3 months.")
        print("Action at the expiry!")
        a = -c + p + S_0
        if S_t > K:
            print("Receive " + str(a * np.exp(r * T)) + " USD from investment, exercise the call to buy the stock for " + str(K)
                  + " USD. Net profit is " + str(a * np.exp(r * T) - K) + " USD.")
        else:
            print("Receive " + str(a * np.exp(r * T)) + " USD from investment, put is exercised and the stock is bought for " + str(K)
                  + " USD. Net profit is " + str(a * np.exp(r * T) - K) + " USD.")
    else:
        print("The call is overvalued.")
        print("Action now!")
        print("Short the call " + str(c) + " USD and buy the put " + str(p) + " USD, buy the stock " 
              + str(S_0) + " USD and borrow " + str(-c + p + S_0) + " USD for 3 months.")
        print("Action at the expiry!")
        a = -c + p + S_0
        if S_t > K:
            print("The call is exercised and the stock is sold for " + str(K) + " USD, use " + str(a * np.exp(r * T)) 
                  + " to repay loan. " + "Net profit is " + str(K - a * np.exp(r * T)) + " USD.")
        else:
            print("Exercise the put to sell stock at " + str(K) + " USD and use " + str(a * np.exp(r * T)) + " to repay loan."
                  + " Net profit is " + str(K - a * np.exp(r * T)) + " USD.")

In [ ]:
# Example for p = 2.25
# Given by questions
S_0 = 31 ## USD
r = 0.1 ## 10%
c = 3 ## USD
K = 30
T = 3/12 ## 3 month
D = 0 ## 0 when no divident

p = 2.25
S_t = 31

# Calculation
PCP(S_0, r, c, K, T, D, p, S_t)

In [ ]:
# Example for p = 1
# Given by questions
S_0 = 31 ## USD
r = 0.1 ## 10%
c = 3 ## USD
K = 30
T = 3/12 ## 3 month
D = 0 ## 0 when no divident

p = 1
S_t = 31

# Calculation
PCP(S_0, r, c, K, T, D, p, S_t)

## Considering dividends

c >= max(S_0 - D - K * e ** (-r * T ; 0) <br>
p >= max(K * e ** (-r * T ; 0) <br>
c + D + K * e ** (-r * T) = p + S <br>

## Early exercise of American options

Without divident, early exercise of call option is not optimal. 
- Insurance
- Time value of cash

Without divident, early exercise of put option is optimal when the put option is in the money.

As to option with divident, more tricky.

# Binomial option pricing (OFOD CH13, 21)

Cox-Ross-Rubinstein binomial model main assumptions:

- one risky and one risk-free asset
- trading is open at discrete times
- the duration between two dates is of length δt
- over one period, two outcomes are possible for the stock price
 - up move
 - down move

## One-step binomial trees

### The replication portfolio approach (No Arbitrage argument)

- S_0: The initial stock price
- f: Option price at t = 0
- T: Time to maturity (オプション満期)
- u: Upward ## u > 1
- d: Downward ## d < 1
- f_u: Option value at maturity when S_0 goes up to S_0 * u
- f_d: Option value at maturity when S_0 goes down to S_0 * d
- delta: portfolio is risk free

From S_0 * u * delta - f_u = S_0 * d * delta - f_d, <br>
delta = (f_u - f_d) / S_0 * (u - d) ...A

Because the initial cost for creating the portfolio should be equal to the current value of portfolio, <br>
S_0 * delta - f = (S_0 * u * delta - f_u) * np.exp(-r * T) ...B

By imputing A into B and rearranging B,<br>
f = np.exp(-r * T) * (p * f_u + (1-p) * f_d), where p = (np.exp(r * T) - d) / (u - d)

In [29]:
# Function of one step binamial tree call given by NA argument
def one_step_binomial_tree_NA(S_0, T, u, d, r, K, call=True):
    if call:
        f_u = np.maximum(S_0 * u - K, 0)
        f_d = np.maximum(S_0 * d - K, 0)
    else:
        f_u = np.maximum(K - S_0 * u, 0)
        f_d = np.maximum(K - S_0 * d, 0)
    delta = (f_u - f_d) / S_0 * (u - d)
    p = (np.exp(r * T) - d) / (u - d)
    f = np.exp(-r * T) * (p * f_u + (1-p) * f_d)
    return f

In [32]:
# Example for one step binomial tree call
# Given by questions
S_0 = 20 ## USD
T = 3/12 ## months
u = 1.1 ## up
d = 0.9 ## down
r = 0.12 ## risk free rate
K = 21

# Answer
one_step_binomial_tree_NA(S_0, T, u, d, r, K, call=True)

0.6329950990317132

### The probabilistic approach (Risk Neutralized argument)

From the perspective of probability, <br>
f = (p * f_u + (1-p) * f_d) * np.exp(-r * T)

In [24]:
# Function of one step binamial tree given by RN argument
def one_step_binomial_tree_RN(S_0, T, u, d, r, K, call=True):
    if call:
        f_u = np.maximum(S_0 * u - K, 0)
        f_d = np.maximum(S_0 * d - K, 0)
    else:
        f_u = np.maximum(K - S_0 * u, 0)
        f_d = np.maximum(K - S_0 * d, 0)
    p = (np.exp(r * T) - d) / (u - d)
    f = (p * f_u + (1-p) * f_d) * np.exp(-r * T)
    return f

In [28]:
# Example for one step binomial tree
# Given by questions
S_0 = 20 ## USD
T = 3/12 ## months
u = 1.1 ## up
d = 0.9 ## down
r = 0.12 ## risk free rate
K = 21

# Answer
one_step_binomial_tree_RN(S_0, T, u, d, r, K, call=True)

0.6329950990317132

## Multi-period binomial trees

### 2 steps

In [48]:
# Function of two step binomial tree
def two_step_binomial_tree(S0, T, u, d, r, K, call=True):
    # Going up or going down at node 1
    p = (np.exp(r*T) - d) / (u-d)
    S0_u = S0 * u
    S0_d = S0 * d
    print("Node 1  S0_u: " + str(S0_u))
    print("Node 1  S0_d: " + str(S0_d))
    
    # Going up or going down at node 2
    S0_uu = S0_u * u
    S0_ud = S0_u * d
    S0_dd = S0_d * d
    print("Node 2  S0_uu: " + str(S0_uu))
    print("Node 2  S0_ud: " + str(S0_ud))
    print("Node 2  S0_dd: " + str(S0_dd))
    
    if call:
        # Call option value at Node 2
        f_uu = np.maximum(S0_uu - K, 0)
        f_ud = np.maximum(S0_ud - K, 0)
        f_dd = np.maximum(S0_dd - K, 0)
        print("Node 2  f_uu: " + str(f_uu))
        print("Node 2  f_ud: " + str(f_ud))
        print("Node 2  f_dd: " + str(f_dd))
        
        # Call option value at Node 1
        f_u = (p * f_uu + (1-p) * f_ud) * np.exp(-r * T)
        f_d = (p * f_ud + (1-p) * f_dd) * np.exp(-r * T)
        print("Node 1  f_u: " + str(f_u))
        print("Node 1  f_d: " + str(f_d))
        
        # Call option value at Node 0
        f = (p * f_u + (1-p) * f_d) * np.exp(-r * T)
        print("Node 0  f: " + str(f))
        
    else:
        # Put option value at Node 2
        f_uu = np.maximum(K - S0_uu, 0)
        f_ud = np.maximum(K - S0_ud, 0)
        f_dd = np.maximum(K - S0_dd, 0)
        print("Node 2  f_uu: " + str(f_uu))
        print("Node 2  f_ud: " + str(f_ud))
        print("Node 2  f_dd: " + str(f_dd))
        
        # Call option value at Node 1
        f_u = (p * f_uu + (1-p) * f_ud) * np.exp(-r * T)
        f_d = (p * f_ud + (1-p) + f_dd) * np.exp(-r * T)
        print("Node 1  f_u: " + str(f_u))
        print("Node 1  f_d: " + str(f_d))
        
        # Call option value at Node 0
        f = (p * f_u + (1-p) * f_d) * np.exp(-r * T)
        print("Node 0  f: " + str(f))
        
    return f

In [51]:
# Example
# given variables
S0 = 20
T = 1/4
u = 1.1
d = 0.9
r = 0.12
K = 21

# Answer
two_step_binomial_tree(S0, T, u, d, r, K, call=True)

Node 1  S0_u: 22.0
Node 1  S0_d: 18.0
Node 2  S0_uu: 24.200000000000003
Node 2  S0_ud: 19.8
Node 2  S0_dd: 16.2
Node 2  f_uu: 3.200000000000003
Node 2  f_ud: 0.0
Node 2  f_dd: 0.0
Node 1  f_u: 2.0255843169014844
Node 1  f_d: 0.0
Node 0  f: 1.2821849452741405


1.2821849452741405

### 4 step

In [70]:
# Function of two step binomial tree
def four_step_binomial_tree(S0, T, u, d, r, K, call=True):
    # Going up or going down at node 1
    p = (np.exp(r*T) - d) / (u-d)
    S0_u = S0 * u
    S0_d = S0 * d
    print("Node 1  S0_u: " + str(S0_u))
    print("Node 1  S0_d: " + str(S0_d))
    
    # Going up or going down at node 2
    S0_u2d0 = S0_u * u
    S0_u1d1 = S0_u * d
    S0_u0d2 = S0_d * d
    print("Node 2  S0_u2d0: " + str(S0_u2d0))
    print("Node 2  S0_u1d1: " + str(S0_u1d1))
    print("Node 2  S0_u0d2: " + str(S0_u0d2))
    
    # Going up or going down at node 3
    S0_u3d0 = S0_u2d0 * u
    S0_u2d1 = S0_u2d0 * d
    S0_u1d2 = S0_u1d1 * d
    S0_u0d3 = S0_u0d2 * d
    print("Node 3  S0_u3d0: " + str(S0_u3d0))
    print("Node 3  S0_u2d1: " + str(S0_u2d1))
    print("Node 3  S0_u1d2: " + str(S0_u1d2))
    print("Node 3  S0_u0d3: " + str(S0_u0d3))
    
    # Going up or going dowm at Node 4
    S0_u4d0 = S0_u3d0 * u
    S0_u3d1 = S0_u3d0 * d
    S0_u2d2 = S0_u2d1 * d
    S0_u1d3 = S0_u1d2 * d
    S0_u0d4 = S0_u0d3 * d
    
    print("Node 3  S0_u4d0: " + str(S0_u4d0))
    print("Node 3  S0_u3d1: " + str(S0_u3d1))
    print("Node 3  S0_u2d2: " + str(S0_u2d2))
    print("Node 3  S0_u1d3: " + str(S0_u1d3))
    print("Node 3  S0_u0d4: " + str(S0_u0d4))
    
    if call:
        
        # Call option value at Node 4
        f_u4d0 = np.maximum(S0_u4d0 - K, 0)
        f_u3d1 = np.maximum(S0_u3d1 - K, 0)
        f_u2d2 = np.maximum(S0_u2d2 - K, 0)
        f_u1d3 = np.maximum(S0_u1d3 - K, 0)
        f_u0d4 = np.maximum(S0_u0d4 - K, 0)
        print("Node 4  f_u4d0: " + str(f_u4d0))
        print("Node 4  f_u3d1: " + str(f_u3d1))
        print("Node 4  f_u2d2: " + str(f_u2d2))
        print("Node 4  f_u1d3: " + str(f_u1d3))
        print("Node 4  f_u0d4: " + str(f_u0d4))
        
        # Call option value at Node 3
        f_u3d0 = (p * f_u4d0 + (1-p) * f_u3d1) * np.exp(-r * T)
        f_u2d1 = (p * f_u3d1 + (1-p) * f_u2d2) * np.exp(-r * T)
        f_u1d2 = (p * f_u2d2 + (1-p) * f_u1d3) * np.exp(-r * T)
        f_u0d3 = (p * f_u1d3 + (1-p) * f_u0d4) * np.exp(-r * T)
        print("Node 3  f_u3d0: " + str(f_u3d0))
        print("Node 3  f_u2d1: " + str(f_u2d1))
        print("Node 3  f_u1d2: " + str(f_u1d2))
        print("Node 3  f_u0d3: " + str(f_u0d3))
        
        # Call option value at Node 2
        f_u2d0 = (p * f_u3d0 + (1-p) * f_u2d1) * np.exp(-r * T)
        f_u1d1 = (p * f_u2d1 + (1-p) * f_u1d2) * np.exp(-r * T)
        f_u0d2 = (p * f_u1d2 + (1-p) * f_u0d3) * np.exp(-r * T)
        print("Node 2  f_u2d0: " + str(f_u2d0))
        print("Node 2  f_u1d1: " + str(f_u1d1))
        print("Node 2  f_u0d2: " + str(f_u0d2))
        
        # Call option value at Node 1
        f_u = (p * f_u2d0 + (1-p) * f_u1d1) * np.exp(-r * T)
        f_d = (p * f_u1d1 + (1-p) * f_u0d2) * np.exp(-r * T)
        print("Node 1  f_u: " + str(f_u))
        print("Node 1  f_d: " + str(f_d))
        
        # Call option value at Node 0
        f = (p * f_u + (1-p) * f_d) * np.exp(-r * T)
        print("Node 0  f: " + str(f))
        
    else:
        # Put option value at Node 4
        f_u4d0 = np.maximum(K - S0_u4d0, 0)
        f_u3d1 = np.maximum(K - S0_u3d1, 0)
        f_u2d2 = np.maximum(K - S0_u2d2, 0)
        f_u1d3 = np.maximum(K - S0_u1d3, 0)
        f_u0d4 = np.maximum(K - S0_u0d4, 0)
        print("Node 4  f_u4d0: " + str(f_u4d0))
        print("Node 4  f_u3d1: " + str(f_u3d1))
        print("Node 4  f_u2d2: " + str(f_u2d2))
        print("Node 4  f_u1d3: " + str(f_u1d3))
        print("Node 4  f_u0d4: " + str(f_u0d4))
        
        # Put option value at Node 3
        f_u3d0 = (p * f_u4d0 + (1-p) * f_u3d1) * np.exp(-r * T)
        f_u2d1 = (p * f_u3d1 + (1-p) * f_u2d2) * np.exp(-r * T)
        f_u1d2 = (p * f_u2d2 + (1-p) * f_u1d3) * np.exp(-r * T)
        f_u0d3 = (p * f_u1d3 + (1-p) * f_u0d4) * np.exp(-r * T)
        print("Node 3  f_u3d0: " + str(f_u3d0))
        print("Node 3  f_u2d1: " + str(f_u2d1))
        print("Node 3  f_u1d2: " + str(f_u1d2))
        print("Node 3  f_u0d3: " + str(f_u0d3))
        
        # Put option value at Node 2
        f_u2d0 = (p * f_u3d0 + (1-p) * f_u2d1) * np.exp(-r * T)
        f_u1d1 = (p * f_u2d1 + (1-p) * f_u1d2) * np.exp(-r * T)
        f_u0d2 = (p * f_u1d2 + (1-p) * f_u0d3) * np.exp(-r * T)
        print("Node 2  f_u2d0: " + str(f_u2d0))
        print("Node 2  f_u1d1: " + str(f_u1d1))
        print("Node 2  f_u0d2: " + str(f_u0d2))
        
        # Put option value at Node 1
        f_u = (p * f_u2d0 + (1-p) * f_u1d1) * np.exp(-r * T)
        f_d = (p * f_u1d1 + (1-p) + f_u0d2) * np.exp(-r * T)
        print("Node 1  f_u: " + str(f_u))
        print("Node 1  f_d: " + str(f_d))
        
        # Put option value at Node 0
        f = (p * f_u + (1-p) * f_d) * np.exp(-r * T)
        print("Node 0  f: " + str(f))

In [71]:
# Example
# given variables
S0 = 20
T = 1/4
u = 1.1
d = 0.9
r = 0.12
K = 21

# Answer
four_step_binomial_tree(S0, T, u, d, r, K, call=True)

Node 1  S0_u: 22.0
Node 1  S0_d: 18.0
Node 2  S0_u2d0: 24.200000000000003
Node 2  S0_u1d1: 19.8
Node 2  S0_u0d2: 16.2
Node 3  S0_u3d0: 26.620000000000005
Node 3  S0_u2d1: 21.780000000000005
Node 3  S0_u1d2: 17.82
Node 3  S0_u0d3: 14.58
Node 3  S0_u4d0: 29.282000000000007
Node 3  S0_u3d1: 23.958000000000006
Node 3  S0_u2d2: 19.602000000000004
Node 3  S0_u1d3: 16.038
Node 3  S0_u0d4: 13.122
Node 4  f_u4d0: 8.282000000000007
Node 4  f_u3d1: 2.9580000000000055
Node 4  f_u2d2: 0.0
Node 4  f_u1d3: 0.0
Node 4  f_u0d4: 0.0
Node 3  f_u3d0: 6.2406437954813345
Node 3  f_u2d1: 1.8723995029358114
Node 3  f_u1d2: 0.0
Node 3  f_u0d3: 0.0
Node 2  f_u2d0: 4.582138963197075
Node 2  f_u1d1: 1.1852197087877845
Node 2  f_u0d2: 0.0
Node 1  f_u: 3.3004244125143116
Node 1  f_d: 0.7502382669384621
Node 0  f: 2.342320707015692


As to n steps, automation by iterating codes is better. We can employ the above code to apply for multi steps version.

## Early exercise of American options

Early exercise should be considered in the calculation for the value of options in each step.

## Calibration of binomial trees (video to come)

### 2 steps

In [92]:
# Function of two step binomial tree calibration
def two_step_binomial_tree_calib(S0, T, sigma, r, K, call=True):
    # Going up or going down at node 1
    dt = T / 2 ## 2 steps
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r*dt) - d) / (u-d)
    S0_u = S0 * u
    S0_d = S0 * d
    print("Node 1  S0_u: " + str(S0_u))
    print("Node 1  S0_d: " + str(S0_d))
    
    # Going up or going down at node 2
    S0_uu = S0_u * u
    S0_ud = S0_u * d
    S0_dd = S0_d * d
    print("Node 2  S0_uu: " + str(S0_uu))
    print("Node 2  S0_ud: " + str(S0_ud))
    print("Node 2  S0_dd: " + str(S0_dd))
    
    if call:
        # Call option value at Node 2
        f_uu = np.maximum(S0_uu - K, 0)
        f_ud = np.maximum(S0_ud - K, 0)
        f_dd = np.maximum(S0_dd - K, 0)
        print("Node 2  f_uu: " + str(f_uu))
        print("Node 2  f_ud: " + str(f_ud))
        print("Node 2  f_dd: " + str(f_dd))
        
        # Call option value at Node 1
        f_u = (p * f_uu + (1-p) * f_ud) * np.exp(-r * dt)
        f_d = (p * f_ud + (1-p) * f_dd) * np.exp(-r * dt)
        print("Node 1  f_u: " + str(f_u))
        print("Node 1  f_d: " + str(f_d))
        
        # Call option value at Node 0
        f = (p * f_u + (1-p) * f_d) * np.exp(-r * dt)
        print("Node 0  f: " + str(f))
        
    else:
        # Put option value at Node 2
        f_uu = np.maximum(K - S0_uu, 0)
        f_ud = np.maximum(K - S0_ud, 0)
        f_dd = np.maximum(K - S0_dd, 0)
        print("Node 2  f_uu: " + str(f_uu))
        print("Node 2  f_ud: " + str(f_ud))
        print("Node 2  f_dd: " + str(f_dd))
        
        # Call option value at Node 1
        f_u = (p * f_uu + (1-p) * f_ud) * np.exp(-r * dt)
        f_d = (p * f_ud + (1-p) + f_dd) * np.exp(-r * dt)
        print("Node 1  f_u: " + str(f_u))
        print("Node 1  f_d: " + str(f_d))
        
        # Call option value at Node 0
        f = (p * f_u + (1-p) * f_d) * np.exp(-r * T)
        print("Node 0  f: " + str(f))
    return p, u, d

In [93]:
# Example
# given variables
S0 = 40
T = 1/2
sigma = 0.3
r = 0.04
K = 40

# Answer
two_step_binomial_tree_calib(S0, T, sigma, r, K, call=True)

Node 1  S0_u: 46.47336970913132
Node 1  S0_d: 34.42831905700231
Node 2  S0_uu: 53.99435230304012
Node 2  S0_ud: 39.99999999999999
Node 2  S0_dd: 29.632728827268714
Node 2  f_uu: 13.994352303040117
Node 2  f_ud: 0.0
Node 2  f_dd: 0.0
Node 1  f_u: 6.8713763591645955
Node 1  f_d: 0.0
Node 0  f: 3.3739191387249114


(0.49594541350546606, 1.161834242728283, 0.8607079764250578)

### 4 step

In [ ]:
# Function of four steps binomial tree calib
def four_step_binomial_tree_calib(S0, T, sigma, r, K, call=True):
    # Going up or going down at node 1
    dt = T / 4 ## 4 steps
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r*dt) - d) / (u-d)
    S0_u = S0 * u
    S0_d = S0 * d
    print("Node 1  S0_u: " + str(S0_u))
    print("Node 1  S0_d: " + str(S0_d))
    
    # Going up or going down at node 2
    S0_u2d0 = S0_u * u
    S0_u1d1 = S0_u * d
    S0_u0d2 = S0_d * d
    print("Node 2  S0_u2d0: " + str(S0_u2d0))
    print("Node 2  S0_u1d1: " + str(S0_u1d1))
    print("Node 2  S0_u0d2: " + str(S0_u0d2))
    
    # Going up or going down at node 3
    S0_u3d0 = S0_u2d0 * u
    S0_u2d1 = S0_u2d0 * d
    S0_u1d2 = S0_u1d1 * d
    S0_u0d3 = S0_u0d2 * d
    print("Node 3  S0_u3d0: " + str(S0_u3d0))
    print("Node 3  S0_u2d1: " + str(S0_u2d1))
    print("Node 3  S0_u1d2: " + str(S0_u1d2))
    print("Node 3  S0_u0d3: " + str(S0_u0d3))
    
    # Going up or going dowm at Node 4
    S0_u4d0 = S0_u3d0 * u
    S0_u3d1 = S0_u3d0 * d
    S0_u2d2 = S0_u2d1 * d
    S0_u1d3 = S0_u1d2 * d
    S0_u0d4 = S0_u0d3 * d
    
    print("Node 3  S0_u4d0: " + str(S0_u4d0))
    print("Node 3  S0_u3d1: " + str(S0_u3d1))
    print("Node 3  S0_u2d2: " + str(S0_u2d2))
    print("Node 3  S0_u1d3: " + str(S0_u1d3))
    print("Node 3  S0_u0d4: " + str(S0_u0d4))
    
    if call:
        
        # Call option value at Node 4
        f_u4d0 = np.maximum(S0_u4d0 - K, 0)
        f_u3d1 = np.maximum(S0_u3d1 - K, 0)
        f_u2d2 = np.maximum(S0_u2d2 - K, 0)
        f_u1d3 = np.maximum(S0_u1d3 - K, 0)
        f_u0d4 = np.maximum(S0_u0d4 - K, 0)
        print("Node 4  f_u4d0: " + str(f_u4d0))
        print("Node 4  f_u3d1: " + str(f_u3d1))
        print("Node 4  f_u2d2: " + str(f_u2d2))
        print("Node 4  f_u1d3: " + str(f_u1d3))
        print("Node 4  f_u0d4: " + str(f_u0d4))
        
        # Call option value at Node 3
        f_u3d0 = (p * f_u4d0 + (1-p) * f_u3d1) * np.exp(-r * dt)
        f_u2d1 = (p * f_u3d1 + (1-p) * f_u2d2) * np.exp(-r * dt)
        f_u1d2 = (p * f_u2d2 + (1-p) * f_u1d3) * np.exp(-r * dt)
        f_u0d3 = (p * f_u1d3 + (1-p) * f_u0d4) * np.exp(-r * dt)
        print("Node 3  f_u3d0: " + str(f_u3d0))
        print("Node 3  f_u2d1: " + str(f_u2d1))
        print("Node 3  f_u1d2: " + str(f_u1d2))
        print("Node 3  f_u0d3: " + str(f_u0d3))
        
        # Call option value at Node 2
        f_u2d0 = (p * f_u3d0 + (1-p) * f_u2d1) * np.exp(-r * dt)
        f_u1d1 = (p * f_u2d1 + (1-p) * f_u1d2) * np.exp(-r * dt)
        f_u0d2 = (p * f_u1d2 + (1-p) * f_u0d3) * np.exp(-r * dt)
        print("Node 2  f_u2d0: " + str(f_u2d0))
        print("Node 2  f_u1d1: " + str(f_u1d1))
        print("Node 2  f_u0d2: " + str(f_u0d2))
        
        # Call option value at Node 1
        f_u = (p * f_u2d0 + (1-p) * f_u1d1) * np.exp(-r * dt)
        f_d = (p * f_u1d1 + (1-p) * f_u0d2) * np.exp(-r * dt)
        print("Node 1  f_u: " + str(f_u))
        print("Node 1  f_d: " + str(f_d))
        
        # Call option value at Node 0
        f = (p * f_u + (1-p) * f_d) * np.exp(-r * dt)
        print("Node 0  f: " + str(f))
        
    else:
        # Put option value at Node 4
        f_u4d0 = np.maximum(K - S0_u4d0, 0)
        f_u3d1 = np.maximum(K - S0_u3d1, 0)
        f_u2d2 = np.maximum(K - S0_u2d2, 0)
        f_u1d3 = np.maximum(K - S0_u1d3, 0)
        f_u0d4 = np.maximum(K - S0_u0d4, 0)
        print("Node 4  f_u4d0: " + str(f_u4d0))
        print("Node 4  f_u3d1: " + str(f_u3d1))
        print("Node 4  f_u2d2: " + str(f_u2d2))
        print("Node 4  f_u1d3: " + str(f_u1d3))
        print("Node 4  f_u0d4: " + str(f_u0d4))
        
        # Put option value at Node 3
        f_u3d0 = (p * f_u4d0 + (1-p) * f_u3d1) * np.exp(-r * dt)
        f_u2d1 = (p * f_u3d1 + (1-p) * f_u2d2) * np.exp(-r * dt)
        f_u1d2 = (p * f_u2d2 + (1-p) * f_u1d3) * np.exp(-r * dt)
        f_u0d3 = (p * f_u1d3 + (1-p) * f_u0d4) * np.exp(-r * dt)
        print("Node 3  f_u3d0: " + str(f_u3d0))
        print("Node 3  f_u2d1: " + str(f_u2d1))
        print("Node 3  f_u1d2: " + str(f_u1d2))
        print("Node 3  f_u0d3: " + str(f_u0d3))
        
        # Put option value at Node 2
        f_u2d0 = (p * f_u3d0 + (1-p) * f_u2d1) * np.exp(-r * dt)
        f_u1d1 = (p * f_u2d1 + (1-p) * f_u1d2) * np.exp(-r * dt)
        f_u0d2 = (p * f_u1d2 + (1-p) * f_u0d3) * np.exp(-r * dt)
        print("Node 2  f_u2d0: " + str(f_u2d0))
        print("Node 2  f_u1d1: " + str(f_u1d1))
        print("Node 2  f_u0d2: " + str(f_u0d2))
        
        # Put option value at Node 1
        f_u = (p * f_u2d0 + (1-p) * f_u1d1) * np.exp(-r * dt)
        f_d = (p * f_u1d1 + (1-p) + f_u0d2) * np.exp(-r * dt)
        print("Node 1  f_u: " + str(f_u))
        print("Node 1  f_d: " + str(f_d))
        
        # Put option value at Node 0
        f = (p * f_u + (1-p) * f_d) * np.exp(-r * dt)
        print("Node 0  f: " + str(f))
        
    return p, u, d

In [91]:
# Example
# given variables
S0 = 40
T = 1/4
sigma = 0.3
r = 0.04
K = 40

# Answer
four_step_binomial_tree_calib(S0, T, sigma, r, K, call=True)

Node 1  S0_u: 43.115366035385264
Node 1  S0_d: 37.10973945314211
Node 2  S0_u2d0: 46.473369709131326
Node 2  S0_u1d1: 40.0
Node 2  S0_u0d2: 34.428319057002305
Node 3  S0_u3d0: 50.09290864767458
Node 3  S0_u2d1: 43.115366035385264
Node 3  S0_u1d2: 37.10973945314211
Node 3  S0_u0d3: 31.940648750375075
Node 3  S0_u4d0: 53.994352303040124
Node 3  S0_u3d1: 46.473369709131326
Node 3  S0_u2d2: 40.0
Node 3  S0_u1d3: 34.428319057002305
Node 3  S0_u0d4: 29.632728827268707
Node 4  f_u4d0: 13.994352303040124
Node 4  f_u3d1: 6.473369709131326
Node 4  f_u2d2: 0.0
Node 4  f_u1d3: 0.0
Node 4  f_u0d4: 0.0
Node 3  f_u3d0: 10.192783751776176
Node 3  f_u2d1: 3.2152411394868623
Node 3  f_u1d2: 0.0
Node 3  f_u0d3: 0.0
Node 2  f_u2d0: 6.672870541424045
Node 2  f_u1d1: 1.5969697467559012
Node 2  f_u0d2: 0.0
Node 1  f_u: 4.1141182531462865
Node 1  f_d: 0.793194743850729
Node 0  f: 2.4406750814031306


(0.4979306672065451, 1.0778841508846315, 0.9277434863285529)

As to n steps, automation by iterating codes is better. We can employ the above code to apply for multi steps version.

### n steps

In [103]:
# Generalize to n steps binomial tree
def binomial_tree_call_put(steps, S_0, T, sigma, r, K, call=True, array_out=False):
    # Init
    dt = T / steps
    u = np.exp(sigma*np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    
    # Price tree
    price_tree = np.zeros([steps+1, steps+1])
    
    for i in range(steps+1):
        for j in range(i+1):
            price_tree[j, i] = S_0 * (d**j)*(u**(i-j))
            
    # Option value
    option = np.zeros([steps+1, steps+1])
    if call:
        option[:, steps] = np.maximum(np.zeros(steps+1), price_tree[:, steps] - K)
    else:
        option[:, steps] = np.maximum(np.zeros(steps+1), K - price_tree[:, steps])
    
    # Calculate option price at t = 0
    for i in np.arange(steps-1, -1, -1):
        for j in np.arange(0, i+1):
            option[j, i] = np.exp(-r * dt)*(p*option[j, i+1]+(1-p)*option[j+1, i+1])
            
    # Return 
    if array_out:
        return [option[0,0], price_tree, option]
    else:
        return option[0,0]

In [1]:
binomial_tree_call_put(steps=2, S_0=50, T=2, sigma= 0.1, r=0.05, K=52, call=False, array_out=True)

NameError: name 'binomial_tree_call_put' is not defined

In [13]:
# Another simplified implementation
def binomial(steps, S_0, T, sigma, r, K, call=True):
    """
    Implements the binomial option pricing model to price a European call option on a stock
    S_0 - stock price today
    K - strike price of the option
    T - time until expiry of the option
    r - risk-free interest rate
    sigma - the volatility of the stock
    steps - number of steps in the model
    """
    dt = T / steps
    u =  np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d)/(u - d)
    C = {}
    
    if call:
        for m in range(0, steps+1):
            C[(steps, m)] = max(S_0 * (u ** (2*m - steps)) - K, 0)
        for k in range(steps-1, -1, -1):
            for m in range(0,k+1):
                C[(k, m)] = np.exp(-r * dt) * (p * C[(k+1, m+1)] + (1-p) * C[(k+1, m)])
    else:
        for m in range(0, steps+1):
            C[(steps, m)] = max(K - S_0 * (u ** (2*m - steps)), 0)
        for k in range(steps-1, -1, -1):
            for m in range(0,k+1):
                C[(k, m)] = np.exp(-r * dt) * (p * C[(k+1, m+1)] + (1-p) * C[(k+1, m)])
                
    return C[(0,0)]

binomial(steps=5000, S_0=1418.15, T=5/12, sigma=0.42, r=0.012, K=1280, call=True)

228.57195116928222